In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/adapter_model.safetensors /content/outputs/checkpoint-2500/
!cp /content/drive/MyDrive/trainer_state.json /content/outputs/checkpoint-2500/

cp: cannot stat '/content/drive/MyDrive/adapter_model.safetensors': No such file or directory
cp: cannot stat '/content/drive/MyDrive/trainer_state.json': No such file or directory


In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("rishiraj/bengalichat", split='train')

data = []
for line in dataset:
    parts = [part.strip() for part in line["text"].split("</s>") if part.strip()]
    if len(parts) >= 3:
        instruction = parts[1][8:].strip()  # Adjust slicing based on dataset inspection
        response = parts[2][13:].strip()    # Adjust slicing based on dataset inspection
        formatted_line = f"Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append({"instruction": instruction, "response": response, "formatted": formatted_line})
    else:
        print(f"Skipping malformed entry: {line['text']}")

# Save to CSV
df = pd.DataFrame(data)
df.to_csv('chat_dataset.csv', index=False, encoding='utf-8')
print(df.head())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

(…)-00000-of-00001-a3cbd3fa0a5ef22f.parquet:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

(…)-00000-of-00001-3b2e30253805f2bd.parquet:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

                                         instruction  \
0  এটি কি ইতিবাচক পর্যালোচনা?\n\nএই ফ্র্যাঞ্চাইজি...   
1  আমি আমার ওয়েবসাইটের জন্য রঙ চয়ন করার চেষ্টা ...   
2  আমি এবং আমার বন্ধু একদিনের জন্য অ্যারিজোনার উই...   
3  ডি অ্যান্ড ডি কী বোঝায়?এখানে এমন কিছু পাঠ্য র...   
4  যেন আপনি স্যার ডেভিড অ্যাটেনবারো ছিলেন, বি-সাই...   

                                            response  \
0  এটি একটি খুব ইতিবাচক পর্যালোচনা, এটি ইঙ্গিত কর...   
1  অবশ্যই, এই কোডটি আপনার যা প্রয়োজন তা করবে।\n\...   
2  উইলিয়ামস, অ্যারিজোনা 1881 সালে বিল উইলিয়ামস ...   
3            ডি অ্যান্ড ডি মানে ডানজিওনস এবং ড্রাগন।   
4  এখানে আমরা তার প্রাকৃতিক আবাসে টি-সাইড প্লেয়া...   

                                           formatted  
0  Instruction:\nএটি কি ইতিবাচক পর্যালোচনা?\n\nএই...  
1  Instruction:\nআমি আমার ওয়েবসাইটের জন্য রঙ চয়...  
2  Instruction:\nআমি এবং আমার বন্ধু একদিনের জন্য ...  
3  Instruction:\nডি অ্যান্ড ডি কী বোঝায়?এখানে এম...  
4  Instruction:\nযেন আপনি স্যার ডেভিড অ্যাটেনবারো..

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("rishiraj/bengalichat", split='train')

data = []
for line in dataset:
    parts = [part.strip() for part in line["text"].split("</s>") if part.strip()]
    if len(parts) >= 3:
        instruction = parts[1][8:].strip()  # Adjust slicing based on dataset inspection
        response = parts[2][13:].strip()    # Adjust slicing based on dataset inspection
        formatted_line = f"Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append({"instruction": instruction, "response": response, "formatted": formatted_line})
    else:
        print(f"Skipping malformed entry: {line['text']}")

# Save to CSV
df = pd.DataFrame(data)
df.to_csv('chat_dataset.csv', index=False, encoding='utf-8')
print(df.head())


                                         instruction  \
0  এটি কি ইতিবাচক পর্যালোচনা?\n\nএই ফ্র্যাঞ্চাইজি...   
1  আমি আমার ওয়েবসাইটের জন্য রঙ চয়ন করার চেষ্টা ...   
2  আমি এবং আমার বন্ধু একদিনের জন্য অ্যারিজোনার উই...   
3  ডি অ্যান্ড ডি কী বোঝায়?এখানে এমন কিছু পাঠ্য র...   
4  যেন আপনি স্যার ডেভিড অ্যাটেনবারো ছিলেন, বি-সাই...   

                                            response  \
0  এটি একটি খুব ইতিবাচক পর্যালোচনা, এটি ইঙ্গিত কর...   
1  অবশ্যই, এই কোডটি আপনার যা প্রয়োজন তা করবে।\n\...   
2  উইলিয়ামস, অ্যারিজোনা 1881 সালে বিল উইলিয়ামস ...   
3            ডি অ্যান্ড ডি মানে ডানজিওনস এবং ড্রাগন।   
4  এখানে আমরা তার প্রাকৃতিক আবাসে টি-সাইড প্লেয়া...   

                                           formatted  
0  Instruction:\nএটি কি ইতিবাচক পর্যালোচনা?\n\nএই...  
1  Instruction:\nআমি আমার ওয়েবসাইটের জন্য রঙ চয়...  
2  Instruction:\nআমি এবং আমার বন্ধু একদিনের জন্য ...  
3  Instruction:\nডি অ্যান্ড ডি কী বোঝায়?এখানে এম...  
4  Instruction:\nযেন আপনি স্যার ডেভিড অ্যাটেনবারো..

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-26 03:46:21 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
# @title To apply Parameter-Efficient Fine-Tuning (PEFT) to the pre-trained Gemma-3-4b model (unsloth provided)
model = FastModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

Unsloth: Making `base_model.model.vision_tower.vision_model` require gradients


In [ ]:
prompt = """
### Instruction:
{}
### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

# Formatting function
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    responses = examples["response"]
    texts = []
    for instruction, response in zip(instructions, responses):
        text = prompt.format(instruction, response) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Reload and format for training
dataset = load_dataset("csv", data_files="chat_dataset.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

{'instruction': 'এটি কি ইতিবাচক পর্যালোচনা?\n\nএই ফ্র্যাঞ্চাইজি কখনও হার্ট স্টপ স্টান্টস, ফাইট সিকোয়েন্স এবং তারকা শক্তি বন্ধ করে হতাশ করে না।এবার মূল চারটি ছাড়াও নতুন মুখগুলি হলেন আবুয়েলিতা চরিত্রে রিতা মোরেনো এবং দান্তে জেসন মোমোয়া যিনি শোটি চুরি করেছেন।', 'response': 'এটি একটি খুব ইতিবাচক পর্যালোচনা, এটি ইঙ্গিত করে যে ফ্র্যাঞ্চাইজি শ্রোতাদের জন্য আরও পছন্দসই চরিত্রগুলি যুক্ত করার পাশাপাশি অতীতের প্রত্যাশার ভিত্তিতে বিতরণ করেছে।', 'formatted': 'Instruction:\nএটি কি ইতিবাচক পর্যালোচনা?\n\nএই ফ্র্যাঞ্চাইজি কখনও হার্ট স্টপ স্টান্টস, ফাইট সিকোয়েন্স এবং তারকা শক্তি বন্ধ করে হতাশ করে না।এবার মূল চারটি ছাড়াও নতুন মুখগুলি হলেন আবুয়েলিতা চরিত্রে রিতা মোরেনো এবং দান্তে জেসন মোমোয়া যিনি শোটি চুরি করেছেন।\n\nResponse:\nএটি একটি খুব ইতিবাচক পর্যালোচনা, এটি ইঙ্গিত করে যে ফ্র্যাঞ্চাইজি শ্রোতাদের জন্য আরও পছন্দসই চরিত্রগুলি যুক্ত করার পাশাপাশি অতীতের প্রত্যাশার ভিত্তিতে বিতরণ করেছে।', 'text': '\n### Instruction:\nএটি কি ইতিবাচক পর্যালোচনা?\n\nএই ফ্র্যাঞ্চাইজি কখনও হার্ট স্টপ স্টান্টস, ফাইট স

In [ ]:
import wandb
!wandb login
wandb.init(project="gemma3-4b-diff-bangla-chat", name="finetuning on chat dataset")

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ankiporel (ankiporel-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

max_seq_length = 2048

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        report_to = "wandb",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/9500 [00:00<?, ? examples/s]

In [ ]:
trainer.train(resume_from_checkpoint="/content/outputs/checkpoint-2500")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,500 | Num Epochs = 3 | Total steps = 3,561
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 148,099,072/4,000,000,000 (3.70% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
2501,3.238100
2502,3.179900
2503,3.710000
2504,2.595000
2505,3.592400
2506,3.413600
2507,2.896100
2508,3.465100
2509,3.018600
2510,2.467100


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=3561, training_loss=0.962387412895282, metrics={'train_runtime': 9593.4326, 'train_samples_per_second': 2.971, 'train_steps_per_second': 0.371, 'total_flos': 3.01938687469882e+17, 'train_loss': 0.962387412895282})

In [ ]:
# @title Save the model
model.save_pretrained("gemma3-4B-pre-ft-v2")
tokenizer.save_pretrained("gemma3-4B-pre-ft-v2")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


['gemma3-4B-pre-ft-v2/processor_config.json']

In [ ]:
# @title Huggingface login
from huggingface_hub import login
login()

In [ ]:
# @title Publish to Huggingface
! huggingface-cli upload Ankita-Porel/gemma3-4B-pre-ft-v2 gemma3-4B-pre-ft-v2

Start hashing 11 files.
Finished hashing 11 files.
  0% 0/3 [00:00<?, ?it/s]
adapter_model.safetensors:   0% 0.00/3.35G [00:00<?, ?B/s]
adapter_model.safetensors:   0% 16.0M/3.35G [00:10<35:11, 1.58MB/s]
adapter_model.safetensors:   1% 32.0M/3.35G [00:12<18:42, 2.95MB/s]
adapter_model.safetensors:   1% 48.0M/3.35G [00:12<11:08, 4.94MB/s]
adapter_model.safetensors:   2% 64.0M/3.35G [00:13<07:12, 7.59MB/s]
adapter_model.safetensors:   3% 96.0M/3.35G [00:13<03:30, 15.4MB/s]
adapter_model.safetensors:   3% 112M/3.35G [00:14<03:13, 16.7MB/s] 
adapter_model.safetensors:   5% 160M/3.35G [00:14<01:31, 34.9MB/s]
adapter_model.safetensors:   6% 192M/3.35G [00:14<01:09, 45.3MB/s]
adapter_model.safetensors:   7% 224M/3.35G [00:14<00:50, 62.3MB/s]
adapter_model.safetensors:   8% 256M/3.35G [00:15<00:45, 68.5MB/s]
adapter_model.safetensors:   9% 288M/3.35G [00:15<00:40, 76.4MB/s]
adapter_model.safetensors:   9% 304M/3.35G [00:15<00:39, 76.1MB/s]
adapter_model.safetensors:  10% 320M/3.35G [00:15<00:3

In [ ]:
# @title Load the model from huggingface for inference
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name="Ankita-Porel/gemma3-4B-pre-ft-v2",
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_8bit = False,
    full_finetuning = False,
    # token="hf_..."
)

In [ ]:
# @title Run the tests
model = FastModel.for_inference(model)

prompt = """
### Instruction:
{}
### Response:
{}
"""

inputs = tokenizer(
[
    prompt.format(
        "পদার্থবিদ্যা কি?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, repetition_penalty = 2.0, use_cache = True)

generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

with open("generated_output.txt", "w", encoding="utf-8") as file:
    file.write(generated_text)

print("Output saved to 'output.txt'")
tokenizer.batch_decode(outputs)